# Entraînement du modèle de classification Call Center sur Google Colab

Ce notebook permet d'entraîner notre modèle de classification des tickets sur Google Colab en utilisant le GPU pour de meilleures performances.

In [ ]:
try:
    from google.colab import drive
except ImportError:
    raise RuntimeError("Ce notebook doit être exécuté dans Google Colab. Veuillez l'ouvrir avec : https://colab.research.google.com")

## 1. Configuration de Google Drive

Premièrement, nous allons connecter Google Drive pour accéder à nos données et sauvegarder nos modèles.

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

# Créer les dossiers nécessaires
!mkdir -p /content/drive/MyDrive/CallCenter/data
!mkdir -p /content/drive/MyDrive/CallCenter/models

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


## 2. Vérification du GPU

Vérifions que nous avons bien accès au GPU sur Colab.

In [ ]:
import torch

# Vérifier si le GPU est disponible
print("GPU disponible:", torch.cuda.is_available())
if torch.cuda.is_available():
    print("Nom du GPU:", torch.cuda.get_device_name(0))
    print("Mémoire totale (Go):", torch.cuda.get_device_properties(0).total_memory / 1e9)

GPU disponible: True
Nom du GPU: Tesla T4
Mémoire totale (Go): 15.828320256


In [ ]:
# Vérification de CUDA
if torch.cuda.is_available():
    print("Version CUDA:", torch.version.cuda)
    print("Nombre de GPUs disponibles:", torch.cuda.device_count())
    # Afficher les informations sur la mémoire GPU
    print("\nMémoire GPU:")
    print("!nvidia-smi")

Version CUDA: 12.6
Nombre de GPUs disponibles: 1

Mémoire GPU:
!nvidia-smi


## 3. Installation des dépendances

Installons les packages nécessaires pour l'entraînement.

In [ ]:
# Installation des packages
!pip install transformers datasets evaluate scikit-learn pandas numpy mlflow --quiet
!pip install accelerate -U --quiet

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 8.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.8/8.8 MB 124.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 102.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 81.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 147.8/147.8 kB 14.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.9/114.9 kB 11.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.0/85.0 kB 8.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 752.6/752.6 kB 57.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 203.4/203.4 kB 16.6 MB/s eta 0:00:00


In [ ]:
# Vérifier les versions installées
import transformers
import datasets
import pandas
import torch

print("transformers version:", transformers.__version__)
print("datasets version:", datasets.__version__)
print("pandas version:", pandas.__version__)
print("torch version:", torch.__version__)

transformers version: 4.57.1
datasets version: 4.0.0
pandas version: 2.2.2
torch version: 2.8.0+cu126


## 4. Chargement et préparation des données

Copions les données nécessaires depuis notre projet local vers Google Drive.

In [ ]:
# Charger les données
import pandas as pd
import os
from sklearn.model_selection import train_test_split
from transformers import AutoTokenizer
from datasets import Dataset

# Vérifier si le fichier existe dans Google Drive
data_path = '/content/drive/MyDrive/CallCenter/data/processed/tickets_clean.csv'
if not os.path.exists(data_path):
    raise FileNotFoundError(
        "\nERREUR : Le fichier tickets_clean.csv n'a pas été trouvé dans Google Drive.\n"
        "Veuillez suivre ces étapes :\n"
        "1. Assurez-vous d'avoir monté Google Drive (exécutez la section 1)\n"
        "2. Créez le dossier : /content/drive/MyDrive/CallCenter/data/\n"
        "3. Uploadez votre fichier tickets_clean.csv dans ce dossier\n"
        "4. Vérifiez que le chemin est correct : " + data_path
    )

print("Chargement des données depuis:", data_path)
df = pd.read_csv(data_path)
print(f"Nombre total d'échantillons chargés : {len(df)}")

# Nettoyage des données
df = df.dropna(subset=['Document', 'Topic_group'])
print(f"Nombre d'échantillons après nettoyage : {len(df)}")

# Préparation des labels
labels = df['Topic_group'].unique()
print(f"\nNombre de classes : {len(labels)}")
print("Classes disponibles :", labels)

label2id = {label: idx for idx, label in enumerate(labels)}
id2label = {idx: label for label, idx in label2id.items()}
df['label'] = df['Topic_group'].map(label2id)

# Split train/test
train_df, test_df = train_test_split(df, test_size=0.2, random_state=42, stratify=df['label'])
print(f"\nSplit des données :")
print(f"- Ensemble d'entraînement : {len(train_df)} échantillons")
print(f"- Ensemble de test : {len(test_df)} échantillons")

# Initialiser le tokenizer
print("\nChargement du tokenizer...")
tokenizer = AutoTokenizer.from_pretrained('distilbert-base-multilingual-cased')

# Fonction de tokenization
def tokenize_function(examples):
    return tokenizer(
        examples['Document'],
        padding='max_length',
        truncation=True,
        max_length=128
    )

# Créer les datasets HF
print("\nCréation des datasets...")
train_dataset = Dataset.from_pandas(train_df[['Document', 'label']])
test_dataset = Dataset.from_pandas(test_df[['Document', 'label']])

print("Tokenisation des données...")
train_tokenized = train_dataset.map(tokenize_function, batched=True)
test_tokenized = test_dataset.map(tokenize_function, batched=True)

# Définir le format pour PyTorch
train_tokenized.set_format('torch', columns=['input_ids', 'attention_mask', 'label'])
test_tokenized.set_format('torch', columns=['input_ids', 'attention_mask', 'label'])

print("\nPréparation des données terminée !")

Chargement des données depuis: /content/drive/MyDrive/CallCenter/data/processed/tickets_clean.csv
Nombre total d'échantillons chargés : 47834
Nombre d'échantillons après nettoyage : 47834

Nombre de classes : 8
Classes disponibles : ['Hardware' 'Access' 'Miscellaneous' 'HR Support' 'Purchase'
 'Administrative rights' 'Storage' 'Internal Project']

Split des données :
- Ensemble d'entraînement : 38267 échantillons
- Ensemble de test : 9567 échantillons

Chargement du tokenizer...

Création des datasets...
Tokenisation des données...


Map:   0%|          | 0/38267 [00:00<?, ? examples/s]

Map:   0%|          | 0/9567 [00:00<?, ? examples/s]


Préparation des données terminée !


## 5. Configuration de l'entraînement avec optimisation mémoire

Configuration du modèle et des paramètres d'entraînement avec les optimisations pour une utilisation efficace du GPU.

In [ ]:
import torch
from transformers import (
    AutoModelForSequenceClassification,
    TrainingArguments,
    Trainer,
    DataCollatorWithPadding
)
from sklearn.metrics import accuracy_score, f1_score

# Configuration du device
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Initialiser le modèle
model = AutoModelForSequenceClassification.from_pretrained(
    'distilbert-base-multilingual-cased',
    num_labels=len(label2id),
    id2label=id2label,
    label2id=label2id
)

# Activer le gradient checkpointing pour économiser la mémoire
model.gradient_checkpointing_enable()

# Déplacer le modèle sur GPU
model = model.to(device)

# Fonction de calcul des métriques
def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    predictions = predictions.argmax(axis=1)

    return {
        'accuracy': accuracy_score(labels, predictions),
        'f1_macro': f1_score(labels, predictions, average='macro'),
        'f1_weighted': f1_score(labels, predictions, average='weighted')
    }

# Configuration de l'entraînement avec optimisations mémoire et stockage
training_args = TrainingArguments(
    output_dir='/content/drive/MyDrive/CallCenter/models/best_model',  # Changé pour sauvegarder directement dans best_model
    eval_strategy='epoch',
    save_strategy='epoch',
    learning_rate=2e-5,
    per_device_train_batch_size=8,  # Réduit de 16 à 8
    per_device_eval_batch_size=8,   # Réduit de 16 à 8
    gradient_accumulation_steps=4,   # Augmenté de 2 à 4
    num_train_epochs=2,             # Réduit de 3 à 2
    weight_decay=0.01,
    load_best_model_at_end=True,
    metric_for_best_model='f1_weighted',
    greater_is_better=True,
    fp16=True,
    gradient_checkpointing=True,
    report_to='none',
    save_total_limit=1,
    push_to_hub=False,
    overwrite_output_dir=True,
    remove_unused_columns=True
)

# Data collator pour le padding dynamique
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

# Initialiser le Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_tokenized,
    eval_dataset=test_tokenized,
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics
)

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-multilingual-cased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/tmp/ipython-input-2922143058.py:63: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


## 6. Entraînement du modèle

Lançons l'entraînement avec les optimisations de mémoire et GPU.

In [ ]:
# Libérer la mémoire cache GPU avant l'entraînement
if torch.cuda.is_available():
    torch.cuda.empty_cache()

# Lancer l'entraînement
trainer.train()

# Évaluation finale
eval_results = trainer.evaluate()
print("\nRésultats de l'évaluation finale:")
print(eval_results)

Step,Training Loss,Validation Loss,Accuracy,F1 Macro,F1 Weighted
100,No log,1.283284,0.597993,0.440001,0.559423
200,No log,0.878258,0.723633,0.671147,0.713505
300,No log,0.667368,0.795338,0.795060,0.794471
400,No log,0.612816,0.804014,0.806465,0.803726
500,0.942000,0.549966,0.822515,0.827770,0.823147
600,0.942000,0.568378,0.814571,0.820149,0.814005
700,0.942000,0.534644,0.820424,0.814376,0.820373
800,0.942000,0.501611,0.837044,0.838475,0.836378
900,0.942000,0.477105,0.839135,0.838776,0.839359
1000,0.504900,0.485938,0.842166,0.842407,0.843220



Résultats de l'évaluation finale:
{'eval_loss': 0.4043479859828949, 'eval_accuracy': 0.8659977004285565, 'eval_f1_macro': 0.8643472774432597, 'eval_f1_weighted': 0.8660024442231926, 'eval_runtime': 13.0537, 'eval_samples_per_second': 732.897, 'eval_steps_per_second': 45.811, 'epoch': 3.0}


In [ ]:
# Fonction pour nettoyer la mémoire GPU
def clear_gpu_memory():
    if torch.cuda.is_available():
        torch.cuda.empty_cache()
        print("Mémoire GPU nettoyée")
        
# Nettoyage avant l'entraînement
clear_gpu_memory()

try:
    # Lancer l'entraînement avec gestion d'erreur
    trainer.train()
except Exception as e:
    print(f"\nErreur pendant l'entraînement : {str(e)}")
    
    # Sauvegarder l'état actuel en cas d'erreur
    print("\nSauvegarde de secours du modèle...")
    trainer.save_model('/content/drive/MyDrive/CallCenter/models/backup_model')
    print("Modèle de secours sauvegardé!")

# Évaluation finale
print("\nÉvaluation du modèle...")
eval_results = trainer.evaluate()
print("\nRésultats de l'évaluation finale:")
print(eval_results)

## 7. Sauvegarde du modèle

Sauvegardons le modèle entraîné sur Google Drive.

In [ ]:
# Chemin de sauvegarde sur Google Drive
save_path = '/content/drive/MyDrive/CallCenter/models/best_model'

# Sauvegarder le modèle et le tokenizer
trainer.save_model(save_path)
tokenizer.save_pretrained(save_path)

# Sauvegarder les mappings label2id et id2label
import json
with open(f'{save_path}/label_mappings.json', 'w') as f:
    json.dump({
        'label2id': label2id,
        'id2label': id2label
    }, f, indent=2)

# ⭐ FORCER LA CRÉATION DU FICHIER pytorch_model.bin
import torch
import os
from transformers import AutoModelForSequenceClassification

print("\n=== Vérification et création du pytorch_model.bin ===")

# Vérifier si le fichier existe déjà
pytorch_model_path = os.path.join(save_path, 'pytorch_model.bin')

if not os.path.exists(pytorch_model_path):
    print("⚠️  Le fichier pytorch_model.bin n'existe pas, création en cours...")
    
    # Récupérer le modèle depuis le trainer
    model = trainer.model
    
    # Gérer les modèles enveloppés (DataParallel, DistributedDataParallel)
    if hasattr(model, 'module'):
        model_to_save = model.module
    else:
        model_to_save = model
    
    # Déplacer sur CPU pour éviter les problèmes mémoire
    model_to_save = model_to_save.to('cpu')
    
    # Sauvegarder le state_dict
    torch.save(model_to_save.state_dict(), pytorch_model_path)
    print(f"✅ Fichier pytorch_model.bin créé avec succès !")
    print(f"   Taille : {os.path.getsize(pytorch_model_path) / (1024*1024):.2f} MB")
else:
    size_mb = os.path.getsize(pytorch_model_path) / (1024*1024)
    print(f"✅ Le fichier pytorch_model.bin existe déjà ! Taille : {size_mb:.2f} MB")

# Vérifier la structure complète du dossier
print("\n📁 Fichiers dans le dossier de sauvegarde :")
for f in sorted(os.listdir(save_path)):
    filepath = os.path.join(save_path, f)
    if os.path.isfile(filepath):
        size = os.path.getsize(filepath) / (1024*1024)
        print(f"   - {f} ({size:.2f} MB)")
    else:
        print(f"   - {f}/ (dossier)")

print("\n✅ Modèle sauvegardé avec succès dans Google Drive !")

Modèle sauvegardé avec succès dans Google Drive !


**Note:** If you encounter a `NameError: name 'trainer' is not defined` when saving the model, ensure that you have executed the "6. Entraînement du modèle" section first. The `trainer` object is created in that section.

In [ ]:
# ⭐ DIAGNOSTIC ET RÉPARATION - Exécuter SI pytorch_model.bin est manquant
import os
import torch
from transformers import AutoModelForSequenceClassification

model_path = '/content/drive/MyDrive/CallCenter/models/best_model/pytorch_model.bin'
save_path = '/content/drive/MyDrive/CallCenter/models/best_model'

print("=== VÉRIFICATION DU MODÈLE ===\n")

# 1. Vérifier l'existence du fichier
if os.path.exists(model_path):
    size_mb = os.path.getsize(model_path) / (1024 * 1024)
    print(f"✅ Le fichier pytorch_model.bin existe ! Taille : {size_mb:.2f} MB")
else:
    print("❌ ERREUR : Le fichier pytorch_model.bin n'existe pas !")
    print("\n🔧 RÉPARATION EN COURS...\n")
    
    # 2. Vérifier ce qui existe dans le dossier
    print("Fichiers trouvés dans le dossier :")
    files = os.listdir(save_path)
    for f in sorted(files):
        print(f"   - {f}")
    
    # 3. Tenter de charger le modèle depuis la configuration
    try:
        print("\n📥 Tentative de chargement du modèle depuis les fichiers existants...")
        model = AutoModelForSequenceClassification.from_pretrained(save_path, local_files_only=True)
        print("✅ Modèle chargé avec succès !")
        
        # 4. Sauvegarder en format unique
        print("\n💾 Sauvegarde en fichier unique...")
        model = model.to('cpu')
        torch.save(model.state_dict(), model_path)
        print(f"✅ Fichier pytorch_model.bin créé ! Taille : {os.path.getsize(model_path) / (1024*1024):.2f} MB")
        
    except Exception as e:
        print(f"\n❌ Erreur : {str(e)}")
        print("\n💡 SOLUTIONS :")
        print("   1. Vérifiez que trainer.save_model() a bien été exécuté")
        print("   2. Vérifiez la sauvegarde dans Google Drive à ce chemin :")
        print(f"      {save_path}")
        print("   3. Sinon, ré-exécutez la cellule de sauvegarde (section 7)")

print("\n" + "="*50)

In [ ]:
# 🧪 TEST DE CHARGEMENT - Vérifier que le modèle fonctionne
from transformers import AutoModelForSequenceClassification, AutoTokenizer

save_path = '/content/drive/MyDrive/CallCenter/models/best_model'

print("🔄 Chargement du modèle depuis Google Drive...\n")

try:
    # Charger le modèle et le tokenizer
    model = AutoModelForSequenceClassification.from_pretrained(save_path)
    tokenizer = AutoTokenizer.from_pretrained(save_path)
    
    print("✅ Modèle chargé avec succès !")
    print(f"   Type : {model.__class__.__name__}")
    print(f"   Nombre de paramètres : {sum(p.numel() for p in model.parameters()) / 1e6:.2f}M")
    
    # Test rapide de prédiction
    print("\n🧪 Test rapide de prédiction...")
    test_text = "Mon problème est résolu"
    
    inputs = tokenizer(test_text, return_tensors="pt", truncation=True, max_length=128)
    outputs = model(**inputs)
    predictions = outputs.logits.argmax(-1)
    
    print(f"   Texte : '{test_text}'")
    print(f"   Classe prédite : {predictions.item()}")
    print("\n✅ Le modèle fonctionne correctement !")
    
except Exception as e:
    print(f"❌ Erreur lors du chargement : {str(e)}")
    print("\n💡 Assurez-vous que pytorch_model.bin existe dans le dossier")

In [ ]:
# Cell à coller dans Colab (exécuter APRES l'entraînement)
import os
import torch
from transformers import AutoModelForSequenceClassification

save_path = "/content/drive/MyDrive/CallCenter/models/best_model"

print("=== Listing du dossier de sauvegarde ===")
if os.path.exists(save_path):
    for f in sorted(os.listdir(save_path)):
        print(f)
else:
    raise FileNotFoundError(f"Le dossier {save_path} n'existe pas. Vérifiez le chemin ou sauvegardez d'abord le modèle.")

# Détecter formats courants
files = os.listdir(save_path)
has_safetensors = any(f.endswith(".safetensors") for f in files)
has_single_bin = "pytorch_model.bin" in files
has_index = "pytorch_model.bin.index.json" in files
shards = [f for f in files if f.startswith("pytorch_model") and f.endswith(".bin") and f != "pytorch_model.bin"]

print("\nDetections:")
print(" - safetensors:", has_safetensors)
print(" - pytorch_model.bin exists:", has_single_bin)
print(" - sharded index json:", has_index)
print(" - shards count:", len(shards))

if has_single_bin or has_safetensors:
    print("\nUn fichier de poids unique est déjà présent. Vous pouvez charger le modèle avec:")
    print(f"AutoModelForSequenceClassification.from_pretrained('{save_path}')")
else:
    print("\nAucun fichier unique trouvé. On va tenter de forcer la création d'un fichier 'pytorch_model.bin' unique.")
    # Récupérer l'objet modèle depuis trainer si possible, sinon recharger depuis le dossier (attention mémoire)
    try:
        model = trainer.model  # si trainer défini dans le notebook
        print(" - Modèle récupéré depuis 'trainer.model'.")
    except NameError:
        model = None
        print(" - 'trainer' non trouvé en scope. On va essayer de recharger depuis le dossier (peut demander beaucoup de RAM).")

    # Si on ne peut pas récupérer trainer.model, on tente from_pretrained (transformers va merger les shards automatiquement si possible)
    if model is None:
        try:
            print(" - Tentative de rechargement depuis le dossier (AutoModelForSequenceClassification.from_pretrained).")
            model = AutoModelForSequenceClassification.from_pretrained(save_path, local_files_only=False)
            print(" - Rechargement OK.")
        except Exception as e:
            raise RuntimeError("Impossible de recharger le modèle depuis le dossier. Message d'erreur: " + str(e))

    # Gérer les modèles distribués/DP
    if hasattr(model, "module"):
        print(" - Détecté model.module (wrapped). On prend .module")
        model_to_save = model.module
    else:
        model_to_save = model

    # Move to CPU pour sauver (sécurise la mémoire GPU)
    device_before = next(model_to_save.parameters()).device
    try:
        model_to_save.to("cpu")
    except Exception as e:
        print(" - Warning: impossible de déplacer le modèle sur CPU:", e)

    # Sauvegarder state_dict en un seul fichier
    target_file = os.path.join(save_path, "pytorch_model.bin")
    print(f"\nSauvegarde de state_dict dans: {target_file}")
    try:
        torch.save(model_to_save.state_dict(), target_file)
        print(" - Sauvegarde réussie.")
    except Exception as e:
        raise RuntimeError("Erreur lors de torch.save: " + str(e))
    finally:
        # remettre sur l'appareil d'origine si possible
        try:
            model_to_save.to(device_before)
        except Exception:
            pass

    print("\nListe finale des fichiers:")
    for f in sorted(os.listdir(save_path)):
        print(f)

    print("\nVous pouvez maintenant charger avec:\nAutoModelForSequenceClassification.from_pretrained('{}')".format(save_path))